In [1]:
!pip uninstall -y transformers accelerate
!pip install -U transformers accelerate bitsandbytes sentencepiece peft datasets trl

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.11.0-py3-none-any.whl (375 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]s]


In [2]:
import transformers
transformers.__version__


'4.57.1'

In [3]:
from huggingface_hub import login
login()


In [4]:
!pip install hf_transfer

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import accelerate
model_name = "google/gemma-3-1b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Fast tokenizer bug → 반드시 use_fast=False
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

save_dir = "./gemma3_1b_base_v2"
tokenizer.save_pretrained(save_dir)
base_model.save_pretrained(save_dir)

print("✅ 모델 저장 완료")


✅ 모델 저장 완료


In [7]:
from datasets import load_dataset

raw_dataset = load_dataset(
    "json",
    data_files="real_final_counseling.jsonl"
)["train"]

raw_dataset[0]


{'messages': [{'role': 'system',
   'content': '당신은 공감적이고 실질적인 진로·심리 상담사입니다. 질문에 대해 구체적이고 현실적인 조언을 제공합니다.'},
  {'role': 'user',
   'content': "최근 친구들이 자신이 좋아하는 것과 잘하는 것을 발견해 진로를 고민하는 모습을 보면서, 나는 그들과는 다르게 아무런 흥미나 재능이 없는 것 같아 많이 괴롭습니다. 매일 학교에서 어떤 과목이 나에게 맞는지 고민하지만, 그럴수록 답답함이 커져만 가요. 이런 상황 속에서 '나는 대체 누구이고, 어떤 특성을 가지고 있는지'에 대한 고민이 더욱 깊어지고 있습니다. 진로는 내 삶의 중요한 방향성을 결정하는 문제인데, 나 자신에 대해 잘 모르니 더 막막한 것 같아요. 이런 상태에서 어떻게 내 특성과 흥미를 발견할 수 있을까요?"},
  {'role': 'assistant',
   'content': "자신의 특성 파악은 진로선택의 가장 기본적인 토대가 되기 때문에 특별히 뭘 하고 싶은지, 무엇에 관심이 있는지 모르고 있다면 많이 답답하고 속상할 수 있어요. 진로를 결정할 때, 잘 하거나 좋아하는 게 없다고 생각될 때만큼 곤혹스러울 때가 없는 것 같아요. 진로란 앞으로의 인생계획과 관계된 것인데 자신의 능력이나 흥미에 대해서 잘 모를 때는 어느 곳으로 방향을 정해야할지 막막하게 느껴지거든요. 하지만 진로는 다른 사람이 대신 찾아줄 수 있는 것이 아니기 때문에 시간이 걸리고 힘들더라도 스스로 내가 가지고 있는 특성이 무엇인지, 나는 어떤 분야에 흥미와 재능이 있는지를 조금 더 구체적으로 발견할 수 있도록 노력하는 것이 필요해요. 우선 '내가 누구인가, 나는 어떤 사람인가, 나는 어떤 특성을 가지고 있는가'를 찾아보고 관심을 가지고 있는 분야가 무엇인지를 탐색해보는 것이 가장 첫 번째 단계라고 할 수 있어요. 그것이 굳이 직업, 학교공부 등과 관련된 것이 아니라고 하더라도 난 이것을 할 때 제일 신났었다, 평소에 친구들과 이런

In [8]:
import re

def clean_text(text):
    text = re.sub(r"<[^>]+>", " ", text)       # HTML 제거
    text = text.replace("&nbsp;", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def strip_outer_quotes(text):
    text = text.strip()
    if len(text) >= 2 and text[0] == '"' and text[-1] == '"':
        return text[1:-1].strip()
    return text

def example_to_text(example):
    msgs = example["messages"]

    system = clean_text(msgs[0]["content"])
    user = clean_text(msgs[1]["content"])
    assistant = strip_outer_quotes(clean_text(msgs[2]["content"]))

    # Gemma-3 공식 포맷 기반 ChatML 구조
    text = (
        "<start_of_turn>system\n"
        f"{system}\n"
        "<end_of_turn>\n"
        "<start_of_turn>user\n"
        f"{user}\n"
        "<end_of_turn>\n"
        "<start_of_turn>assistant\n"
        f"{assistant}\n"
        "<end_of_turn>"
    )
    return {"text": text}

dataset = raw_dataset.map(example_to_text)
dataset[0]


{'messages': [{'role': 'system',
   'content': '당신은 공감적이고 실질적인 진로·심리 상담사입니다. 질문에 대해 구체적이고 현실적인 조언을 제공합니다.'},
  {'role': 'user',
   'content': "최근 친구들이 자신이 좋아하는 것과 잘하는 것을 발견해 진로를 고민하는 모습을 보면서, 나는 그들과는 다르게 아무런 흥미나 재능이 없는 것 같아 많이 괴롭습니다. 매일 학교에서 어떤 과목이 나에게 맞는지 고민하지만, 그럴수록 답답함이 커져만 가요. 이런 상황 속에서 '나는 대체 누구이고, 어떤 특성을 가지고 있는지'에 대한 고민이 더욱 깊어지고 있습니다. 진로는 내 삶의 중요한 방향성을 결정하는 문제인데, 나 자신에 대해 잘 모르니 더 막막한 것 같아요. 이런 상태에서 어떻게 내 특성과 흥미를 발견할 수 있을까요?"},
  {'role': 'assistant',
   'content': "자신의 특성 파악은 진로선택의 가장 기본적인 토대가 되기 때문에 특별히 뭘 하고 싶은지, 무엇에 관심이 있는지 모르고 있다면 많이 답답하고 속상할 수 있어요. 진로를 결정할 때, 잘 하거나 좋아하는 게 없다고 생각될 때만큼 곤혹스러울 때가 없는 것 같아요. 진로란 앞으로의 인생계획과 관계된 것인데 자신의 능력이나 흥미에 대해서 잘 모를 때는 어느 곳으로 방향을 정해야할지 막막하게 느껴지거든요. 하지만 진로는 다른 사람이 대신 찾아줄 수 있는 것이 아니기 때문에 시간이 걸리고 힘들더라도 스스로 내가 가지고 있는 특성이 무엇인지, 나는 어떤 분야에 흥미와 재능이 있는지를 조금 더 구체적으로 발견할 수 있도록 노력하는 것이 필요해요. 우선 '내가 누구인가, 나는 어떤 사람인가, 나는 어떤 특성을 가지고 있는가'를 찾아보고 관심을 가지고 있는 분야가 무엇인지를 탐색해보는 것이 가장 첫 번째 단계라고 할 수 있어요. 그것이 굳이 직업, 학교공부 등과 관련된 것이 아니라고 하더라도 난 이것을 할 때 제일 신났었다, 평소에 친구들과 이런

In [9]:
def preprocess(examples):
    tok = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=2048
    )

    # LM labels
    tok["labels"] = tok["input_ids"]

    # Gemma-3 필수: token_type_ids 직접 생성
    tok["token_type_ids"] = [
        [0] * len(x) for x in tok["input_ids"]
    ]

    return tok

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.remove_columns(["text"])
dataset[0]


{'messages': [{'role': 'system',
   'content': '당신은 공감적이고 실질적인 진로·심리 상담사입니다. 질문에 대해 구체적이고 현실적인 조언을 제공합니다.'},
  {'role': 'user',
   'content': "최근 친구들이 자신이 좋아하는 것과 잘하는 것을 발견해 진로를 고민하는 모습을 보면서, 나는 그들과는 다르게 아무런 흥미나 재능이 없는 것 같아 많이 괴롭습니다. 매일 학교에서 어떤 과목이 나에게 맞는지 고민하지만, 그럴수록 답답함이 커져만 가요. 이런 상황 속에서 '나는 대체 누구이고, 어떤 특성을 가지고 있는지'에 대한 고민이 더욱 깊어지고 있습니다. 진로는 내 삶의 중요한 방향성을 결정하는 문제인데, 나 자신에 대해 잘 모르니 더 막막한 것 같아요. 이런 상태에서 어떻게 내 특성과 흥미를 발견할 수 있을까요?"},
  {'role': 'assistant',
   'content': "자신의 특성 파악은 진로선택의 가장 기본적인 토대가 되기 때문에 특별히 뭘 하고 싶은지, 무엇에 관심이 있는지 모르고 있다면 많이 답답하고 속상할 수 있어요. 진로를 결정할 때, 잘 하거나 좋아하는 게 없다고 생각될 때만큼 곤혹스러울 때가 없는 것 같아요. 진로란 앞으로의 인생계획과 관계된 것인데 자신의 능력이나 흥미에 대해서 잘 모를 때는 어느 곳으로 방향을 정해야할지 막막하게 느껴지거든요. 하지만 진로는 다른 사람이 대신 찾아줄 수 있는 것이 아니기 때문에 시간이 걸리고 힘들더라도 스스로 내가 가지고 있는 특성이 무엇인지, 나는 어떤 분야에 흥미와 재능이 있는지를 조금 더 구체적으로 발견할 수 있도록 노력하는 것이 필요해요. 우선 '내가 누구인가, 나는 어떤 사람인가, 나는 어떤 특성을 가지고 있는가'를 찾아보고 관심을 가지고 있는 분야가 무엇인지를 탐색해보는 것이 가장 첫 번째 단계라고 할 수 있어요. 그것이 굳이 직업, 학교공부 등과 관련된 것이 아니라고 하더라도 난 이것을 할 때 제일 신났었다, 평소에 친구들과 이런

In [10]:
import torch
from transformers import DataCollatorForLanguageModeling

class GemmaCollator(DataCollatorForLanguageModeling):
    def __call__(self, features):
        batch = super().__call__(features)

        if "token_type_ids" not in features[0]:
            batch["token_type_ids"] = torch.zeros_like(batch["input_ids"])
        else:
            batch["token_type_ids"] = torch.tensor(
                [f["token_type_ids"] for f in features],
                dtype=torch.long
            )
        return batch

data_collator = GemmaCollator(tokenizer=tokenizer, mlm=False)


In [11]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=32,                     # 64 가능하나 1B 모델은 16~32 추천
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
)


In [25]:
import os
os.environ["WANDB_DISABLED"] = "true"

from trl import SFTConfig, SFTTrainer

train_config = SFTConfig(
    output_dir="gemma3_lora_output4",
    num_train_epochs=10,                  # 1 → 2로 추천
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_32bit",
)

trainer = SFTTrainer(
    model=base_model,
    args=train_config,
    train_dataset=dataset,
    data_collator=data_collator,
    peft_config=lora_config,
    processing_class=tokenizer
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
U

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"

from trl import SFTConfig, SFTTrainer
from transformers import TrainerCallback

# 커스텀 EarlyStopping (train loss 기반)
class CustomEarlyStopping(TrainerCallback):
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            current_loss = logs['loss']
            
            # 개선이 있는 경우
            if current_loss < self.best_loss - self.min_delta:
                self.best_loss = current_loss
                self.counter = 0
                print(f"\n✅ Loss improved to {current_loss:.4f}")
            else:
                self.counter += 1
                print(f"\n⚠️  No improvement (patience: {self.counter}/{self.patience})")
                
                # patience 초과 시 학습 중단
                if self.counter >= self.patience:
                    print(f"\n🛑 Early stopping triggered! Best loss: {self.best_loss:.4f}")
                    control.should_training_stop = True
        
        return control


train_config = SFTConfig(
    output_dir="gemma3_lora_output3",
    num_train_epochs=20,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    
    # 로깅 설정
    logging_steps=20,
    logging_first_step=True,
    report_to=["none"],
    
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_32bit",
    disable_tqdm=False,
)

# 콜백들
early_stopping = CustomEarlyStopping(patience=10, min_delta=0.001)
printing_callback = PrintingCallback()

trainer = SFTTrainer(
    model=base_model,
    args=train_config,
    train_dataset=dataset,
    data_collator=data_collator,
    peft_config=lora_config,
    processing_class=tokenizer,
    callbacks=[early_stopping, printing_callback],
)

PyTorch: setting up devices
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Using auto half precision backend


In [26]:
trainer.train()


The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: token_type_ids, messages, attention_mask. If token_type_ids, messages, attention_mask are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,046
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 6,310
  Number of trainable parameters = 26,091,520
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more d

Step,Training Loss
10,2.312800
20,1.479200
30,1.363600
40,1.260500
50,1.191500
60,1.142500
70,1.216600
80,1.127500
90,1.083700
100,1.165500


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Saving model checkpoint to gemma3_lora_output4/checkpoint-631
loading configuration file config.json from cache at /workspace/.cache/huggingface/hub/models--google--gemma-3-1b-it/snapshots/dcc83ea841ab6100d6b47a070329e1ba4cf78752/config.json
Model config Gemma3TextConfig {
  "_sliding_window_pattern": 6,
  "architectures": [
    "Gemma3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": null,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "dtype": "bfloat16",
  "eos_token_id": [
    1,
    106
  ],
  "final_logit_softcapping": null,
  "head_dim": 256,
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "layer_types": [
    "sliding_attention",
    "sliding_

TrainOutput(global_step=6310, training_loss=0.656725995060759, metrics={'train_runtime': 13162.453, 'train_samples_per_second': 3.834, 'train_steps_per_second': 0.479, 'total_flos': 2.2445514847420416e+17, 'train_loss': 0.656725995060759, 'epoch': 10.0})

In [ ]:
trainer.model.save_pretrained("gemma3_lora_college2")
tokenizer.save_pretrained("gemma3_lora_college2")


In [ ]:
from huggingface_hub import create_repo

create_repo(
    repo_id="rlawnsrb731/gemma3-finetuned-interview",
    private=False   # ← True로 하면 비공개
)


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    repo_id="seungok00/gemma3-finetuned-interview",
    folder_path="gemma3_lora_interview",
    commit_message="Upload finetuned LoRA model"
)


In [ ]:
from peft import PeftModel

ft_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

ft_model = PeftModel.from_pretrained(
    ft_model,
    "gemma3_lora_interview"
)

print("✅ Finetuned model loaded!")


In [ ]:
def infer_base(q):
    base_model.eval()  # ✅ inference 모드 강제
    inp = tokenizer(q, return_tensors="pt")
    
    inp = inp.to(base_model.device)
    
    # token_type_ids는 generate()에 전달하지 않음 (Gemma-3는 지원하지 않음)
    # generate()에 필요한 키만 전달
    generate_kwargs = {
        "input_ids": inp["input_ids"],
        "attention_mask": inp.get("attention_mask", None)
    }
    # None 값 제거
    generate_kwargs = {k: v for k, v in generate_kwargs.items() if v is not None}

    out = base_model.generate(
        **generate_kwargs,
        max_new_tokens=200
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)


def infer_ft(q):
    ft_model.eval()  # ✅ inference 모드 강제
    inp = tokenizer(q, return_tensors="pt")
    
    inp = inp.to(ft_model.device)
    
    # token_type_ids는 generate()에 전달하지 않음 (Gemma-3는 지원하지 않음)
    # generate()에 필요한 키만 전달
    generate_kwargs = {
        "input_ids": inp["input_ids"],
        "attention_mask": inp.get("attention_mask", None)
    }
    # None 값 제거
    generate_kwargs = {k: v for k, v in generate_kwargs.items() if v is not None}

    out = ft_model.generate(
        **generate_kwargs,
        max_new_tokens=200
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)


In [ ]:
import pandas as pd

eval_df = pd.DataFrame({
    "question": [
        next(m["content"] for m in rec["messages"] if m["role"] == "user")
        for rec in raw_dataset
    ],
    "ground_truth": [
        next(m["content"] for m in rec["messages"] if m["role"] == "assistant")
        for rec in raw_dataset
    ]
})

eval_df.head()


In [ ]:
from tqdm import tqdm

def build_ragas_df(df, infer_fn):
    data = {"question":[], "answer":[], "ground_truth":[], "contexts":[]}

    for i in tqdm(range(len(df))):
        q  = df.iloc[i]["question"]
        gt = df.iloc[i]["ground_truth"]

        pred = infer_fn(q)

        data["question"].append(q)
        data["answer"].append(pred)
        data["ground_truth"].append(gt)
        data["contexts"].append([])  # 문서 기반 RAG 아니라 빈 리스트 유지

    return pd.DataFrame(data)


In [ ]:
pip install ragas

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

df_base = build_ragas_df(eval_df, infer_base)
df_ft   = build_ragas_df(eval_df, infer_ft)

base_scores = evaluate(df_base, metrics=[faithfulness(), answer_relevancy()])
ft_scores   = evaluate(df_ft, metrics=[faithfulness(), answer_relevancy()])

print("✅ Base model RAGAS:", base_scores)
print("✅ Finetuned model RAGAS:", ft_scores)



In [ ]:
compare = pd.DataFrame({
    "metric": base_scores.keys(),
    "base": base_scores.values(),
    "finetuned": ft_scores.values(),
})

compare["delta"] = compare["finetuned"] - compare["base"]
compare
